### Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import matplotlib.patches as mpatches
import matplotlib.lines as mlines
from matplotlib.ticker import AutoMinorLocator, MultipleLocator

import ADT_OT2_Analysis as AD

### Variables

Membrane Thickness:

| Sample | Before (mm) | After (mm) |
| --- | --- | --- |
| S1 |  |  |
| S2 |  |  |
| S3 |  |  |
| S4 |  |  |

In [2]:
# Testing Details

Date = 'YYYYMMDD'
Membrane = 'membrane'
Dye_name = 'Dye_name'
dye_sh = 'dye_shorthand'
dilution_factor = 10

# Membrane sample names (in order of H1, H2, H3, H4)
# membrane_L is membrane thickness (in microns). Default is 110
H_cells = { }
 
H_cells['H1'] = {'sample':'S1', 'membrane_L': 110}
H_cells['H2'] = {'sample':'S2', 'membrane_L': 110}
H_cells['H3'] = {'sample':'S3', 'membrane_L': 110}
H_cells['H4'] = {'sample':'S4', 'membrane_L': 110}

In [3]:
#experiment_handle = '20250412_ot2_PES_P40G00_RB'
experiment_handle = Date + '_ot2_PES_' + Membrane + '_' + dye_sh

# Files to read
calibration_curve_csv = 'OT2_Data/'+ Date + '_ot2_calibration_' + dye_sh + '.csv'
longform_filename = 'OT2_Data/' + experiment_handle + '_long.csv'
shortform_filename = 'OT2_Data/' + experiment_handle + '_short.csv'

# What you want to name the files that will be saved
longform_save_name = 'Results/' + experiment_handle + '_metadata.csv'
full_df_save_name ='Results/' + experiment_handle + '.csv'

# Diffusion parameters
total_vol = 2.2e13 #(um^3)
area = np.pi*(16000/2)**2 #(um^2)

In [3]:
# Labels for plot legends
plot_H1 = Membrane + ' ' + H_cells['H1']['sample']
plot_H2 = Membrane + ' ' + H_cells['H2']['sample']
plot_H3 = Membrane + ' ' + H_cells['H3']['sample']
plot_H4 = Membrane + ' ' + H_cells['H4']['sample']
plot_labels = [plot_H1, plot_H2, plot_H3, plot_H4]

max_t = 50 #max time (hrs)

# Colors by dye type
rb = 'palevioletred'
mb = 'rebeccapurple'
bb = 'royalblue'
ao = 'darkorange'
contrast = 'dimgrey'
DT_suite = [rb, mb, bb, ao]

dye = bb

# comparison options
contrast_A = 'cornflowerblue'
contrast_B = 'forestgreen'
contrast_C = 'violet'

#colorblind suite
color_A = (0,0.306,0.920) #blue
color_B = (0,0.5,0) #green
color_C = (0.73,0.33,0.83) #fuschia
color_D = (0.88,0.55,0) #orange
CB_suite = [color_A, color_B, color_C, color_D]

# Original suite
OG_A = 'cornflowerblue'
OG_B = 'forestgreen'
OG_C = 'peru'
OG_D = 'palevioletred'
OG_suite = [OG_A, OG_B, OG_C, OG_D]

## Calibration Curve

typical abs slope per dye:

| Dye | Abs | or |
| --- | --- | --- |
| BB | 0.069 | 0.0723 | 
| AO | 0.0135 |  |
| RB | 0.0589 | 0.058 |
| MB | 0.0327 | 0.035 |

In [ ]:
#absorption coefficient dertermined from Calibration section
m, b = AD.absorptivity(calibration_curve_csv)
absorptivity = round(m,4)

print('absorptivity = ', absorptivity)

In [ ]:
# plotting the calibration curve for a sanity check (optional)
calibration_curve_df = pd.read_csv(calibration_curve_csv)
X = calibration_curve_df['C_uM']
Y = calibration_curve_df['Abs']
yfit = [b + m * xi for xi in X]

fig, ax = plt.subplots(figsize=(3.5,3.5))

ax.plot(X, Y, ms=6, marker='o', linewidth=0, color='black')
ax.plot(X, yfit, linewidth=2, color = 'darkorange')
ax.set_xlabel(r'Concentration ($\mu M$)', fontsize=12)
ax.set_ylabel('Abs', fontsize=12)
ax.tick_params(labelsize=11)

plt.tight_layout()
plt.show();

## Analysis

### Part 1: Longform .csv

In [ ]:
longform = AD.longform_record(longform_filename, absorptivity, dilution_factor)
longform

### Save the metadata file:

In [ ]:
longform.to_csv(longform_save_name, index=False)

### Part 2: Shortform .csv (Concentrations, Averages, and Diffusion Coefficients)
This is where the shortform file is converted to a new df with the original dye concentrations (instead of UV-Vis absorbances)

The concentration data for each sample is then used to determine:
- Diffusion coefficients calculated at each time point per membrane sample
- overall diffusion coefficient for each individual sample, averaged over the given index range

In [ ]:
full_df, D_aves = AD.data_calculations(shortform_filename, absorptivity, dilution_factor, 
                     total_vol, area, H_cells=H_cells, index_range=[1,2,3,4,5,6,7,8,9,10,11])

In [ ]:
full_df

In [ ]:
D_aves

If each sample is behaving differently and you want to calculate the averages separately, use the following:

In [ ]:
H1_index_range = [1,2,3,4,5,6,7,8,9,10,11]
H2_index_range = [1,2,3,4,5,6,7,8,9,10,11]
H3_index_range = [1,2,3,4,5,6,7,8,9,10,11]
H4_index_range = [1,2,3,4,5,6,7,8,9,10,11]

H1_D_ave = full_df[H_cells['H1']['sample']].iloc[H1_index_range].mean()
H2_D_ave = full_df[H_cells['H2']['sample']].iloc[H2_index_range].mean()
H3_D_ave = full_df[H_cells['H3']['sample']].iloc[H3_index_range].mean()
H4_D_ave = full_df[H_cells['H4']['sample']].iloc[H4_index_range].mean()
H1_D_std = full_df[H_cells['H1']['sample']].iloc[H1_index_range].std()
H2_D_std = full_df[H_cells['H2']['sample']].iloc[H2_index_range].std()
H3_D_std = full_df[H_cells['H3']['sample']].iloc[H3_index_range].std()
H4_D_std = full_df[H_cells['H4']['sample']].iloc[H4_index_range].std()

print(str(H_cells['H1']['sample']), '=', H1_D_ave.round(3), '+/-', H1_D_std.round(3))
print(str(H_cells['H2']['sample']), '=', H2_D_ave.round(3), '+/-', H2_D_std.round(3))
print(str(H_cells['H3']['sample']), '=', H3_D_ave.round(3), '+/-', H3_D_std.round(3))
print(str(H_cells['H4']['sample']), '=', H4_D_ave.round(3), '+/-', H4_D_std.round(3))

### Save the final DF

In [ ]:
full_df.to_csv(full_df_save_name, index=False)

# Plots

### Concentration vs Time

If using all 8 H-cells, use the `AD.progress_plots_8` function instead.

In [ ]:
AD.progress_plots_4(full_df, sample_names=H_cells, max_time=max_t, max_D = 60, color_set=OG_suite, font=18)

### Total Concentration vs Time

Sum the concentrations in Cells 1 and 2 to keep track of total C (in case of partitioning/evaporation)

In [ ]:
sum_df = pd.DataFrame(data=full_df)
sum_df['H1_sum'] = sum_df['H1_C1']+sum_df['H1_C2']
sum_df['H2_sum'] = sum_df['H2_C1']+sum_df['H2_C2']
sum_df['H3_sum'] = sum_df['H3_C1']+sum_df['H3_C2']
sum_df['H4_sum'] = sum_df['H4_C1']+sum_df['H4_C2']


fig, ax = plt.subplots(figsize=(6,5))

ax.plot(sum_df['Time'], sum_df['H1_sum'], label=H_cells['H1']['sample'], ms=8, marker='o',
        color=CB_suite[0], linewidth=0)
ax.plot(sum_df['Time'], sum_df['H2_sum'], label=H_cells['H2']['sample'], ms=8, marker='o',
        color=CB_suite[1], linewidth=0)
ax.plot(sum_df['Time'], sum_df['H3_sum'], label=H_cells['H3']['sample'], ms=8, marker='o',
        color=CB_suite[2], linewidth=0)
ax.plot(sum_df['Time'], sum_df['H4_sum'], label=H_cells['H4']['sample'], ms=8, marker='o',
        color=CB_suite[3], linewidth=0)
ax.set_xlabel('Time (hr)', fontsize=16)
ax.set_ylabel(r'Concentration ($\mu M$)', fontsize=16)
#ax.set_ylim(0,df['H1_C1'][0]+25)
ax.set_ylim(0,450)
ax.set_xlim(-2,70)
ax.legend(loc='upper right', ncols=2, fontsize=12, edgecolor='inherit')
ax.tick_params(labelsize=16)

plt.tight_layout()
#plt.savefig('../Figures/'+ Date + 'fig_name.csv', dpi=300)
plt.show();

### Comparison Plot

Compare two (or more) samples from different experiments

In [ ]:
df1 = pd.read_csv('Results/20250603_manual_PES_P25G00_AO.csv')
df2 = pd.read_csv('Results/20250505_ot2_PES_P25G00_AO.csv')

fig2, (ax3,ax4) = plt.subplots(nrows=1, ncols=2, figsize=(12,6), layout='constrained')
#fig2.suptitle('Manual vs Automated Sampling', fontsize=20)

donor = mlines.Line2D([], [], color='black', ls='', marker='v',
                          markersize=8, label='Donor Chamber (Cell 1)')
receptor = mlines.Line2D([], [], color='black', ls='', marker='s',
                          markersize=8, label='Receptor Chamber (Cell 2)')
diffusivity = mlines.Line2D([], [], color='black', ls='', marker='o',
                          markersize=8, label='Diffusion Coefficient')

sample_1 = mpatches.Patch(color='#332288', label='By Hand')
sample_2 = mpatches.Patch(color='#44AA99', label='Automated')


ax3.plot(df2['Time'], df2['H3_C1']/df2['H3_C1'][0], label='Automated (Cell 1)', ms=9, 
         marker='v', color='#44AA99', linewidth=0)
ax3.plot(df2['Time'], df2['H3_C2']/df2['H3_C1'][0], label='Automated (Cell 2)', ms=9, 
         marker='s', color='#44AA99', linewidth=0)
ax3.plot(df1['Time'], df1['H1_C1']/df1['H1_C1'][0], label='By Hand (Cell 1)',
         ms=9, marker='v', color='#332288', linewidth=0)
ax3.plot(df1['Time'], df1['H1_C2']/df1['H1_C1'][0], label='By Hand (Cell 2)',
         ms=9, marker='s', color='#332288', linewidth=0)

ax3.set_xlabel('Time (hr)', fontsize=18)
ax3.set_ylabel('RB Concentration (normalized)', fontsize=18)
ax3.set_ylim(0,1.1)
ax3.set_xlim(-2,70)
ax3.xaxis.set_minor_locator(MultipleLocator(2))
ax3.legend(handles=[donor, receptor], fontsize=14, edgecolor='inherit')
#ax3.legend(loc='upper right', ncols=1, fontsize=16, edgecolor='inherit')
ax3.tick_params(labelsize=18)


ax4.plot(df2['Time'], df2['S3'], label='Automated (P25G00)', ms=10, marker='o', color='#44AA99', linewidth=0)
ax4.plot(df1['Time'], df1['S1'], label='By Hand (P25G00)', ms=10, marker='o', color='#332288', linewidth=0)
ax4.set_xlabel('Time (hr)', fontsize=18)
ax4.set_ylabel('Diffusivity ($\mu m^2/s$)', fontsize=18)
ax4.set_ylim(0,60)
ax4.set_xlim(0,70)
ax4.xaxis.set_minor_locator(MultipleLocator(2))
ax4.legend(handles=[diffusivity], fontsize=14, edgecolor='inherit')
#ax4.legend(loc='upper right', ncols=1, fontsize=16, edgecolor='inherit')
ax4.tick_params(labelsize=18)

fig2.legend(handles=[sample_1, sample_2], loc='outside lower center', ncols=4,
           fontsize=16, edgecolor='inherit')

#plt.tight_layout()
plt.show();